In [29]:
import astropy
from astropy.coordinates import EarthLocation, SkyCoord, AltAz, ICRS
from astropy.time import Time
from astropy.constants import c
import astropy.units as u
import numpy as np

alt = 57.66 * u.deg  # Altitude
az = 179.99 * u.deg   # Azimuth

green_bank = EarthLocation(lat=38.4330 * u.deg, lon=-79.8397 * u.deg, height=800 * u.m)
jodrell_bank = EarthLocation(lat=53.234329 * u.deg, lon=-2.3050403 * u.deg, height=77 * u.m)

obstime = Time('2024-07-18T09:50:54')

altaz = AltAz(alt=alt, az=az, location=jodrell_bank, obstime=obstime)

skycoord = SkyCoord(altaz)

ra = skycoord.icrs.ra
dec = skycoord.icrs.dec

print(ra.deg,dec.deg)

ra_str = ra.to_string(unit=u.hour, sep=':', pad=True)
dec_str = dec.to_string(unit=u.deg, sep=':', pad=True)

print(f'RA: {ra_str}')
print(f'Dec: {dec_str}')

bary_corr = (skycoord.transform_to(ICRS)).radial_velocity_correction(kind='heliocentric')
barry_corr = bary_corr.to(u.km/u.s)

#print radial velocity correction for barycentre
print(f'Radial Velocity correction to barycentre: {barry_corr}')

81.76574635338442 20.87305399347512
RA: 05:27:03.77912481
Dec: 20:52:22.99437651
Radial Velocity correction to barycentre: 16.30662787728845 km / s
